In [29]:
import os
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pickle
import random
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')

In [30]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [31]:
class Config:
    """Configuration class for model parameters"""
    BASE_PATH = r"D:\Projects\Cardiac Patient Monitoring System\vitalldb"
    SIGNAL_NAMES = ["HR", "SBP", "DBP", "MBP", "SpO2", "RR"]
    FILE_KEYWORDS = {
        "HR": "Solar8000_HR.csv",
        "SBP": "Solar8000_ART_SBP.csv",
        "DBP": "Solar8000_ART_DBP.csv",
        "MBP": "Solar8000_ART_MBP.csv",
        "SpO2": "Solar8000_PLETH_SPO2.csv",
        "RR": "Solar8000_VENT_RR.csv"
    }
    SEQUENCE_LEN = 60
    BATCH_SIZE = 32 
#     BATCH_SIZE = 16  # Reduced for memory efficiency
    LEARNING_RATE = 0.001
    NUM_EPOCHS = 15  # Reduced for faster training
    HIDDEN_SIZE = 64  # Reduced for efficiency
    DROPOUT_RATE = 0.3 #DROPOUT_RATE = 0.3<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [32]:
class PatientDataset(Dataset):
    """Custom dataset for patient sequences"""
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [33]:
class ImprovedLSTMClassifier(nn.Module):
    """Improved LSTM classifier with better architecture"""
    def __init__(self, input_size, hidden_size, num_classes=2, dropout_rate=0.3):
        super().__init__()
        
        # Bidirectional LSTM for better pattern recognition
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=2,
            dropout=dropout_rate,
            batch_first=True,
            bidirectional=True  # Better for capturing patterns
        )
        
        # Attention mechanism for better sequence understanding
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size * 2,  # *2 because bidirectional
            num_heads=4,
            dropout=dropout_rate,
            batch_first=True
        )
        
        # Classification head
        self.classifier = nn.Sequential(
            nn.LayerNorm(hidden_size * 2),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, num_classes)
        )
        
    def forward(self, x):
        # LSTM forward pass
        lstm_out, _ = self.lstm(x)  # [batch, seq_len, hidden_size * 2]
        
        # Self-attention to focus on important time steps
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        
        # Global average pooling instead of just last timestep
        pooled = torch.mean(attn_out, dim=1)  # [batch, hidden_size * 2]
        
        # Classification
        output = self.classifier(pooled)
        return output

In [34]:
# class FocalLoss(nn.Module):
#     """Focal Loss for handling class imbalance"""
#     def __init__(self, alpha=None, gamma=2.0):
#         super().__init__()
#         self.alpha = alpha
#         self.gamma = gamma
        
#     def forward(self, inputs, targets):
#         ce_loss = nn.functional.cross_entropy(inputs, targets, reduction='none')
#         pt = torch.exp(-ce_loss)
#         focal_loss = (1 - pt) ** self.gamma * ce_loss
        
#         if self.alpha is not None:
#             alpha_t = self.alpha[targets]
#             focal_loss = alpha_t * focal_loss
            
#         return focal_loss.mean()

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, smoothing=0.05):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.smoothing = smoothing

    def forward(self, inputs, targets):
        num_classes = inputs.size(1)
        true_dist = torch.zeros_like(inputs)
        true_dist.fill_(self.smoothing / (num_classes - 1))
        true_dist.scatter_(1, targets.unsqueeze(1), 1.0 - self.smoothing)

        ce_loss = -(true_dist * torch.log_softmax(inputs, dim=1)).sum(dim=1)
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.alpha is not None:
            alpha_t = self.alpha[targets]
            focal_loss = alpha_t * focal_loss

        return focal_loss.mean()

In [76]:
def load_and_preprocess_data(config: Config) -> Tuple[np.ndarray, np.ndarray]:
    """Load and preprocess patient data with improved memory management"""
    
    # Load clinical data
    clinic_path = os.path.join(config.BASE_PATH, "clinical_info.csv")
    clinic = pd.read_csv(clinic_path)
    
    # Create risk labels if not exists
    if "risk_label" not in clinic.columns:
        def assign_risk(row):
            if row['death_inhosp'] == 1:
                return 2  # Critical
            elif row['icu_days'] >= 2 or row['asa'] >= 3:
                return 1  # Moderate
            else:
                return 0  # Stable
        clinic['risk_label'] = clinic.apply(assign_risk, axis=1)
    
    print(f" Risk label distribution: {dict(clinic['risk_label'].value_counts())}")
    
    # Data holders
    all_sequences = []
    all_labels = []
    
    # Process patients in batches to manage memory
    for idx, case_row in enumerate(tqdm(clinic.itertuples(), desc="Processing patients")):
        caseid = f"case{int(case_row.caseid):04d}"
        label = case_row.risk_label
        
        # Load all signal files for this patient
        patient_dfs = []
        skip_patient = False
        
        for signal, keyword in config.FILE_KEYWORDS.items():
            file_path = os.path.join(config.BASE_PATH, f"{caseid}_{keyword}")
            if os.path.exists(file_path):
                try:
                    df = pd.read_csv(file_path, names=["Time", signal])
                    df["Time"] = pd.to_numeric(df["Time"], errors='coerce')
                    df = df.dropna(subset=["Time"])
                    if len(df) > 0:
                        patient_dfs.append(df)
                    else:
                        skip_patient = True
                        break
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
                    skip_patient = True
                    break
            else:
                skip_patient = True
                break
        
        if skip_patient or len(patient_dfs) != len(config.SIGNAL_NAMES):
            continue
        
        # Merge signals based on time
        merged_df = patient_dfs[0]
        for df in patient_dfs[1:]:
            merged_df = pd.merge_asof(
                merged_df.sort_values("Time"),
                df.sort_values("Time"),
                on="Time",
                direction='nearest'
            )
        
        # Handle missing values
        merged_df = merged_df.fillna(method='ffill').fillna(method='bfill')
        merged_df = merged_df.drop(columns=["Time"])
        
        # Skip if too short
        if len(merged_df) < config.SEQUENCE_LEN + 1:
            continue
        
        # Normalize data
        scaler = StandardScaler()  # Better than MinMax for medical data
        scaled_data = scaler.fit_transform(merged_df.values)
        
        # Create sequences with stride to reduce memory usage
        stride = max(1, config.SEQUENCE_LEN // 4)  # Overlapping sequences
        for i in range(0, len(scaled_data) - config.SEQUENCE_LEN, stride):
            sequence = scaled_data[i:i + config.SEQUENCE_LEN]
            all_sequences.append(sequence)
            all_labels.append(label)
    
    print(f" Total sequences created: {len(all_sequences)}")
    
    # Convert to numpy arrays
    X = np.array(all_sequences, dtype=np.float32)
    y = np.array(all_labels, dtype=np.int64)

    y= np.where(y == 2, 1, 0)
    return X, y,scaler

In [77]:
def balance_dataset(X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Smart balancing strategy for imbalanced dataset"""
    
    print("Original class distribution:", Counter(y))
    
    # Separate classes
    indices_0 = np.where(y == 0)[0]
    indices_1 = np.where(y == 1)[0]
    indices_2 = np.where(y == 2)[0]
    
    # Strategic sampling to balance computational efficiency with performance
    # More balanced approach - not too aggressive undersampling
    n_class_0 = min(len(indices_0), 8000)  # Increased from 2000
    n_class_1 = len(indices_1)  # Keep all class 1
    n_class_2 = len(indices_2)  # Keep all class 2
    
    # Sample indices
    sampled_0 = np.random.choice(indices_0, n_class_0, replace=False)
    sampled_1 = indices_1
    sampled_2 = indices_2
    
    # Combine samples
    X_balanced = np.concatenate([X[sampled_0], X[sampled_1], X[sampled_2]])
    y_balanced = np.concatenate([y[sampled_0], y[sampled_1], y[sampled_2]])
    
    # Data augmentation for minority classes
    X_aug = []
    y_aug = []
    
    # More aggressive augmentation for class 2 (critical cases)
    augment_factors = {0: 0.0, 1: 2.0} # Augment class 1 by 30%, class 2 by 200%
#     augment_factors = {1: 0.3, 2: 2.0} <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<befor edit
#     for target_class in [1, 2]<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<befor edit
    for target_class in [0 , 1]: # 
        class_indices = np.where(y_balanced == target_class)[0]
        class_data = X_balanced[class_indices]
        
        # Calculate number of augmented samples needed
        n_augment = int(len(class_data) * augment_factors[target_class])
        
        if n_augment > 0:
            aug_indices = np.random.choice(len(class_data), n_augment, replace=True)
            
            for idx in aug_indices:
                original_sample = class_data[idx]
                
                # Multiple augmentation techniques
                augmented_samples = []
                
                # 1. Gaussian noise
                noise = np.random.normal(0, 0.02, original_sample.shape)
                augmented_samples.append(original_sample + noise)
                
                # 2. Scaling
                scale_factor = np.random.uniform(0.95, 1.05)
                augmented_samples.append(original_sample * scale_factor)
                
                # 3. Slight time shift (roll)
                shift = np.random.randint(-2, 3)
                augmented_samples.append(np.roll(original_sample, shift, axis=0))
                
                # FIX: Choose one augmentation randomly using integer index
                chosen_idx = np.random.randint(0, len(augmented_samples))
                chosen_augment = augmented_samples[chosen_idx]
                X_aug.append(chosen_augment)
                y_aug.append(target_class)
    
    # Combine original and augmented data
    if X_aug:
        X_aug = np.array(X_aug)
        y_aug = np.array(y_aug)
        X_final = np.concatenate([X_balanced, X_aug])
        y_final = np.concatenate([y_balanced, y_aug])
    else:
        X_final = X_balanced
        y_final = y_balanced
    
    print("Final class distribution:", Counter(y_final))
    return X_final, y_final


In [78]:
def create_weighted_sampler(y_train: np.ndarray) -> WeightedRandomSampler:
    """Create weighted sampler for training"""
    class_counts = Counter(y_train)
    total_samples = len(y_train)
    
    # Calculate weights inversely proportional to class frequency
    weights = {cls: total_samples / count for cls, count in class_counts.items()}
    
    # Create sample weights
    sample_weights = [weights[int(label)] for label in y_train]
    
    return WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )

In [79]:
def train_model(model: nn.Module, train_loader: DataLoader, val_loader: DataLoader, 
                config: Config) -> nn.Module:
    """Training loop with early stopping and validation"""
    
    # Pre-calculate class weights more efficiently
    y_train_all = []
    print("Calculating class weights...")
    for i, (_, labels) in enumerate(train_loader):
        y_train_all.extend(labels.numpy())
        if i % 100 == 0:
            print(f"Processed {i+1} batches...")
    
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(y_train_all),
        y=y_train_all
    )
    class_weights = torch.FloatTensor(class_weights).to(config.DEVICE)
    #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<befor edit
    
    
#     class_weights = np.array(class_weights)

#     adjustment = np.array([1.5, 1.2])
#     class_weights = class_weights * adjustment  # ضرب numpy array في numpy array

#     class_weights = torch.FloatTensor(class_weights).to(config.DEVICE)

    
    print(f"Class weights: {class_weights}")
    
    # Loss function and optimizer
    criterion = FocalLoss(alpha=class_weights, gamma=2.0)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=0.04) #weight_decay=0.01
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.NUM_EPOCHS)
    
    # Training metrics
    best_val_f1 = 0.0
    patience = 5
    patience_counter = 0
    
    for epoch in range(config.NUM_EPOCHS):
        # Training phase
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels = []
        
        train_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{config.NUM_EPOCHS}")
        for batch_idx, (batch_X, batch_y) in enumerate(train_bar):
            batch_X = batch_X.to(config.DEVICE)
            batch_y = batch_y.to(config.DEVICE)
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            train_loss += loss.item()
            train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_labels.extend(batch_y.cpu().numpy())
            
            # Update progress bar
            train_bar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        # Validation phase
        model.eval()
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{config.NUM_EPOCHS}")
            for batch_X, batch_y in val_bar:
                batch_X = batch_X.to(config.DEVICE)
                batch_y = batch_y.to(config.DEVICE)
                
                outputs = model(batch_X)
                val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_labels.extend(batch_y.cpu().numpy())
        
        # Calculate metrics
        train_f1 = f1_score(train_labels, train_preds, average='weighted')
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        val_balanced_acc = balanced_accuracy_score(val_labels, val_preds)
        
        # Per-class metrics for validation
        val_f1_per_class = f1_score(val_labels, val_preds, average=None)
        
        print(f"\nEpoch {epoch+1}/{config.NUM_EPOCHS} Results:")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}, Train F1: {train_f1:.4f}")
        print(f"  Val F1: {val_f1:.4f}, Val Balanced Acc: {val_balanced_acc:.4f}")
        print(f"  Val F1 per class: [Class 0: {val_f1_per_class[0]:.3f}, Class 1: {val_f1_per_class[1]:.3f}]")
        
        # Check class distribution in predictions
        val_pred_dist = Counter(val_preds)
        print(f"  Val predictions distribution: {val_pred_dist}")
        
        # Early stopping
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            patience_counter = 0
            torch.save(model.state_dict(), "best_cardiac_model.pth")
            print("  New best model saved!")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"  Early stopping after {epoch+1} epochs")
                break
        
        scheduler.step()
    
    # Load best model
    model.load_state_dict(torch.load("best_cardiac_model.pth"))
    return model


In [80]:
def evaluate_model(model: nn.Module, test_loader: DataLoader, config: Config):
    """Comprehensive model evaluation"""
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for batch_X, batch_y in tqdm(test_loader, desc="Evaluating"):
            batch_X = batch_X.to(config.DEVICE)
            batch_y = batch_y.to(config.DEVICE)
            
            outputs = model(batch_X)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(outputs, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    # Calculate metrics
    f1_weighted = f1_score(all_labels, all_preds, average='weighted')
    f1_macro = f1_score(all_labels, all_preds, average='macro')
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    
    print("\n FINAL EVALUATION RESULTS:")
    print(f"F1 Score (Weighted): {f1_weighted:.4f}")
    print(f"F1 Score (Macro): {f1_macro:.4f}")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    
    # Detailed classification report
    print("\n Classification Report:")
    target_names = ['Non-Critical (0)', 'Critical (1)']
    print(classification_report(all_labels, all_preds, target_names=target_names, digits=4))
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    print("\n Confusion Matrix:")
    print(cm)
    
    return {
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'balanced_accuracy': balanced_acc,
        'predictions': all_preds,
        'labels': all_labels,
        'probabilities': all_probs
    }

In [82]:
def main():
    """Main execution function"""
    config = Config()
    
    print(" Starting Cardiac Risk Prediction Model Training")
    print(f"Device: {config.DEVICE}")
    
    # Load and preprocess data
    print("\n Loading and preprocessing data...")
    X, y, scaler = load_and_preprocess_data(config)
    
    # Balance dataset
    print("\n Balancing dataset...")
    X_balanced, y_balanced = balance_dataset(X, y)
    
    # Split data
    print("\n Splitting data...")
    X_train, X_test, y_train, y_test = train_test_split(
        X_balanced, y_balanced, 
        test_size=0.2, 
        stratify=y_balanced, 
        random_state=42
    )
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, 
        test_size=0.2, 
        stratify=y_train, 
        random_state=42
    )
    
    print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
    print(f"Train distribution: {Counter(y_train)}")
    print(f"Test distribution: {Counter(y_test)}")
    
    # Create datasets and data loaders
    train_dataset = PatientDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
    val_dataset = PatientDataset(torch.FloatTensor(X_val), torch.LongTensor(y_val))
    test_dataset = PatientDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))
    
    # Create weighted sampler for training
    weighted_sampler = create_weighted_sampler(y_train)
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=config.BATCH_SIZE, 
        sampler=weighted_sampler,
        pin_memory=False,  # Disable for CPU
        num_workers=0  # Fix Windows multiprocessing issue
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=config.BATCH_SIZE, 
        shuffle=False,
        pin_memory=False,  # Disable for CPU
        num_workers=0  # Fix Windows multiprocessing issue
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=config.BATCH_SIZE, 
        shuffle=False,
        pin_memory=False,  # Disable for CPU
        num_workers=0  # Fix Windows multiprocessing issue
    )
    
    # Initialize model
    print("\n🧠 Initializing model...")
    model = ImprovedLSTMClassifier(
        input_size=len(config.SIGNAL_NAMES),
        hidden_size=config.HIDDEN_SIZE,
        num_classes=2,
        dropout_rate=config.DROPOUT_RATE
    ).to(config.DEVICE)
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")
    
    # Train model
    print("\n Training model...")
    model = train_model(model, train_loader, val_loader, config)
    
    # Evaluate model
    print("\n Evaluating model...")
    results = evaluate_model(model, test_loader, config)
    
    import joblib
    
    os.makedirs(r"D:\Projects\Cardiac Patient Monitoring System\models\lstm", exist_ok=True)
    MODEL_PATH = r"D:\Projects\Cardiac Patient Monitoring System\models\lstm\model.pth"
    SCALER_PATH = r"D:\Projects\Cardiac Patient Monitoring System\models\lstm\scaler.pkl"

    torch.save(model.state_dict(), MODEL_PATH)
    joblib.dump(scaler, SCALER_PATH)
    print(f"\n Model saved to {MODEL_PATH}")
    print(f" Scaler saved to {SCALER_PATH}")

    # Save results
    with open('cardiac_model_results.pkl', 'wb') as f:
        pickle.dump(results, f)
    
    print("\n Training completed successfully!")
    print(f"Best F1 Score: {results['f1_weighted']:.4f}")
    print(f"Best Balanced Accuracy: {results['balanced_accuracy']:.4f}")

if __name__ == "__main__":
    main()

 Starting Cardiac Risk Prediction Model Training
Device: cpu

 Loading and preprocessing data...
 Risk label distribution: {0: 5425, 1: 906, 2: 57}


Processing patients: 6388it [00:49, 128.11it/s] 


 Total sequences created: 245506

 Balancing dataset...
Original class distribution: Counter({0: 245026, 1: 480})
Final class distribution: Counter({0: 8000, 1: 1440})

 Splitting data...
Train: 6041, Val: 1511, Test: 1888
Train distribution: Counter({0: 5119, 1: 922})
Test distribution: Counter({0: 1600, 1: 288})

🧠 Initializing model...
Total parameters: 210,882

 Training model...
Calculating class weights...
Processed 1 batches...
Processed 101 batches...
Class weights: tensor([0.9766, 1.0246])


Validation Epoch 1/15: 100%|███████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 81.61it/s]



Epoch 1/15 Results:
  Train Loss: 0.1540, Train F1: 0.6365
  Val F1: 0.8345, Val Balanced Acc: 0.8602
  Val F1 per class: [Class 0: 0.877, Class 1: 0.600]
  Val predictions distribution: Counter({0: 1028, 1: 483})
  New best model saved!


...


Epoch 14/15 Results:
  Train Loss: 0.0171, Train F1: 0.9922
  Val F1: 0.9883, Val Balanced Acc: 0.9930
  Val F1 per class: [Class 0: 0.993, Class 1: 0.962]
  Val predictions distribution: Counter({0: 1263, 1: 248})
  New best model saved!


Validation Epoch 15/15: 100%|██████████████████████████████████████████████████████████| 48/48 [00:00<00:00, 72.70it/s]



Epoch 15/15 Results:
  Train Loss: 0.0179, Train F1: 0.9929
  Val F1: 0.9883, Val Balanced Acc: 0.9930
  Val F1 per class: [Class 0: 0.993, Class 1: 0.962]
  Val predictions distribution: Counter({0: 1263, 1: 248})

 Evaluating model...


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 72.34it/s]



 FINAL EVALUATION RESULTS:
F1 Score (Weighted): 0.9824
F1 Score (Macro): 0.9667
Balanced Accuracy: 0.9880

 Classification Report:
                  precision    recall  f1-score   support

Non-Critical (0)     0.9994    0.9794    0.9893      1600
    Critical (1)     0.8969    0.9965    0.9441       288

        accuracy                         0.9820      1888
       macro avg     0.9481    0.9880    0.9667      1888
    weighted avg     0.9837    0.9820    0.9824      1888


 Confusion Matrix:
[[1567   33]
 [   1  287]]

 Model saved to D:\Projects\Cardiac Patient Monitoring System\models\lstm\model.pth
 Scaler saved to D:\Projects\Cardiac Patient Monitoring System\models\lstm\scaler.pkl

 Training completed successfully!
Best F1 Score: 0.9824
Best Balanced Accuracy: 0.9880
